In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Input
from keras.layers import concatenate
from keras.optimizers import SGD
from keras.layers.core import Lambda
import keras.backend as K

In [4]:
LSTM_output_size=4
banch_size_lstm=100

## SGD network

LSTM+全连接

In [23]:
# 考虑1000天，50只股票，100个微观因子，10个宏观因子
# SGD networks
macro_in1 = Input(shape=(10,))
macro_in1_1 = Lambda(lambda x:K.reshape(x, shape=(-1,1, 10)))(macro_in1)


sgd_lstm_out = LSTM(4)(macro_in1_1) # (1000, 4)
sgd_lstm_out = Lambda(lambda x:K.repeat(x, 50))(sgd_lstm_out)

micro_in1 = Input(shape=(50,100)) 
return_next_date1 = Input(shape=(50,))

sgd_weights_input = concatenate([micro_in1, sgd_lstm_out]) # (1000, 50, 104)  

sgd_weights_output = Dense(64, activation='relu')(sgd_weights_input)
sgd_weights_output = Dense(64, activation='relu')(sgd_weights_input)
sgd_weights_output = Dense(1)(sgd_weights_input)
sgd_weights_output = Lambda(lambda x:K.reshape(x, shape=(-1, 50)))(sgd_weights_output)


def construction(x):
    tmp = x[0] * x[1]
    tmp = 1-K.sum(tmp, axis=1)
    tmp = K.reshape(tmp, shape=(-1,1)) # (1000, 1)
    tmp = K.repeat(tmp, 50) # (1000, 50, 1)
    tmp = K.reshape(tmp, shape=(-1,50)) # (1000, 50)
    tmp = tmp * x[1] # (1000, 50)
    tmp = K.reshape(tmp, shape=(-1, 50, 1))
    return tmp # the M_{t+1}R_{t+1}

sgd_construction_ouput = Lambda(construction)([sgd_weights_output,return_next_date1])


## attain con_g_output 
con_g_output_loaded = Input(shape=(50, 8))

loss_function_w = Lambda(lambda x:x[0]*x[1])([sgd_construction_ouput, con_g_output_loaded]) # (1000,50, 8)
loss_function_w = Lambda(lambda x:K.reshape(x, shape=(-1, 400)))(loss_function_w)

model_output_w = Model(inputs=[macro_in1, micro_in1], outputs=sgd_weights_output) # acquires weights given info

model_output_sgd = Model(inputs=[macro_in1, micro_in1, return_next_date1], outputs=sgd_construction_ouput) # acquires MR for condition networks

model_w = Model(inputs=[macro_in1, micro_in1, return_next_date1, con_g_output_loaded], outputs=loss_function_w)
model_w.compile(optimizer='adam', loss='mean_squared_error')




In [26]:
# conditional networks
macro_in2 = Input(shape=(10,))
macro_in2_1 = Lambda(lambda x:K.reshape(x, shape=(-1,1, 10)))(macro_in2)

micro_in2 = Input(shape=(50,100))

return_next_date2 = Input(shape=(50,))

sgd_construction_ouput_loaded = Input(shape=(50,1))

con_lstm_out = LSTM(4)(macro_in2_1) # (1000, 4)
con_lstm_out = Lambda(lambda x:K.repeat(x, 50))(con_lstm_out)

con_weights_input = concatenate([micro_in2, con_lstm_out]) # (1000, 50, 104)  

con_g_output = Dense(64, activation='relu')(con_weights_input)
con_g_output = Dense(8)(con_g_output) # (1000, 50, 8)

loss_function_g = Lambda(lambda x:1/(0.001+x[0]*x[1]))([sgd_construction_ouput_loaded, con_g_output]) # (1000,50, 8)
loss_function_g = Lambda(lambda x:K.reshape(x, shape=(-1, 400)))(loss_function_g)

model_output_g = Model(inputs=[macro_in2, micro_in2], outputs=con_g_output) # acquires MR for condition networks

model_g = Model(inputs=[macro_in2, micro_in2, return_next_date2, sgd_construction_ouput_loaded], outputs=loss_function_g)
model_g.compile(optimizer='adam', loss='mean_squared_error')

In [27]:
y_train = np.random.random((1000, 400))
macro_in = np.random.random((1000, 10))
micro_in = np.random.random((1000, 50, 100))
return_next_date = np.random.random((1000, 50))
function_g = np.random.random((1000, 50, 8))
for i in range(2):
    # SDF nets
    model_w.fit([macro_in, micro_in, return_next_date, function_g], y_train, epochs=10, batch_size=200)
    function_sgd = model_output_sgd.predict([macro_in, micro_in, return_next_date])
    print(function_sgd)
    # conditional nets
    model_g.fit([macro_in, micro_in, return_next_date, function_sgd], y_train, epochs=10, batch_size=200)
    function_g = model_output_g.predict([macro_in, micro_in])
    
# get the optimal weights(SDF weights)
print(model_output_w.predict([macro_in, micro_in])[0])

Epoch 1/10
100/100 [==============================] - 1s 15ms/step - loss: 0.3143
Epoch 2/10
100/100 [==============================] - 1s 11ms/step - loss: 0.3105
Epoch 3/10
100/100 [==============================] - 1s 9ms/step - loss: 0.3067
Epoch 4/10
100/100 [==============================] - 1s 9ms/step - loss: 0.3029
Epoch 5/10
100/100 [==============================] - 1s 10ms/step - loss: 0.2992
Epoch 6/10
100/100 [==============================] - 1s 10ms/step - loss: 0.2956
Epoch 7/10
100/100 [==============================] - 1s 10ms/step - loss: 0.2920
Epoch 8/10
100/100 [==============================] - 1s 11ms/step - loss: 0.2885
Epoch 9/10
100/100 [==============================] - 1s 12ms/step - loss: 0.2850
Epoch 10/10
100/100 [==============================] - 1s 11ms/step - loss: 0.2816


In [ ]:
y_train = np.random.random(1)

In [ ]:
y_train

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# 期望输入数据尺寸: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # 返回维度为 32 的向量序列
model.add(LSTM(32, return_sequences=True))  # 返回维度为 32 的向量序列
model.add(LSTM(32))  # 返回维度为 32 的单个向量
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# 生成虚拟训练数据
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# 生成虚拟验证数据
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=5,
          validation_data=(x_val, y_val))

In [17]:
K=np.random.random((1000,10))
K

array([[0.90353493, 0.20216126, 0.41790339, ..., 0.55039814, 0.86175544,
        0.70928886],
       [0.32452567, 0.85630184, 0.59458935, ..., 0.09543519, 0.46461841,
        0.27331959],
       [0.62028556, 0.49443296, 0.15008156, ..., 0.99144689, 0.32100579,
        0.06468506],
       ...,
       [0.08456323, 0.42261279, 0.98871538, ..., 0.29013578, 0.82997586,
        0.2987058 ],
       [0.66707212, 0.50337306, 0.88517786, ..., 0.30469212, 0.70286074,
        0.74676035],
       [0.92523623, 0.15166534, 0.70512259, ..., 0.54618122, 0.41346263,
        0.03624436]])

In [30]:
K.shape

(1000, 10)

In [19]:
KK=np.repeat(K,50)
KK

array([0.90353493, 0.90353493, 0.90353493, ..., 0.03624436, 0.03624436,
       0.03624436])

In [20]:
KK.shape

(500000,)

In [21]:
K

array([[0.90353493, 0.20216126, 0.41790339, ..., 0.55039814, 0.86175544,
        0.70928886],
       [0.32452567, 0.85630184, 0.59458935, ..., 0.09543519, 0.46461841,
        0.27331959],
       [0.62028556, 0.49443296, 0.15008156, ..., 0.99144689, 0.32100579,
        0.06468506],
       ...,
       [0.08456323, 0.42261279, 0.98871538, ..., 0.29013578, 0.82997586,
        0.2987058 ],
       [0.66707212, 0.50337306, 0.88517786, ..., 0.30469212, 0.70286074,
        0.74676035],
       [0.92523623, 0.15166534, 0.70512259, ..., 0.54618122, 0.41346263,
        0.03624436]])

In [31]:
KKK=K.reshape((1,-1,10))
KKK

array([[[0.90353493, 0.20216126, 0.41790339, ..., 0.55039814,
         0.86175544, 0.70928886],
        [0.32452567, 0.85630184, 0.59458935, ..., 0.09543519,
         0.46461841, 0.27331959],
        [0.62028556, 0.49443296, 0.15008156, ..., 0.99144689,
         0.32100579, 0.06468506],
        ...,
        [0.08456323, 0.42261279, 0.98871538, ..., 0.29013578,
         0.82997586, 0.2987058 ],
        [0.66707212, 0.50337306, 0.88517786, ..., 0.30469212,
         0.70286074, 0.74676035],
        [0.92523623, 0.15166534, 0.70512259, ..., 0.54618122,
         0.41346263, 0.03624436]]])

In [32]:
KKK.shape

(1, 1000, 10)

In [30]:
K=np.random.random((1000,50))
K

array([[0.99628178, 0.29435117, 0.46284603, ..., 0.30222297, 0.81661866,
        0.91125386],
       [0.58166571, 0.82097276, 0.29561252, ..., 0.07252075, 0.93790192,
        0.23404779],
       [0.24662393, 0.8790145 , 0.0437841 , ..., 0.90069746, 0.26071325,
        0.80119435],
       ...,
       [0.0704563 , 0.74303609, 0.33504429, ..., 0.37740743, 0.89038861,
        0.48007831],
       [0.58790177, 0.65143357, 0.31676697, ..., 0.88985375, 0.45117266,
        0.54780127],
       [0.52528805, 0.41007237, 0.81842603, ..., 0.63310726, 0.89992533,
        0.4285128 ]])

In [33]:
np.sum(K,axis=0).shape

(50,)

In [34]:
np.sum(K,axis=1).shape

(1000,)

In [19]:
KK=np.random.random((1,50))
KK

array([[0.77975869, 0.93603067, 0.79986745, 0.04857298, 0.73845586,
        0.02749797, 0.87503324, 0.53976727, 0.05298363, 0.3511038 ,
        0.96704053, 0.91207224, 0.0372148 , 0.45494143, 0.64264947,
        0.70155295, 0.62358242, 0.00407979, 0.39574575, 0.71317738,
        0.9508727 , 0.38351257, 0.02334646, 0.57094746, 0.66164877,
        0.12517307, 0.41672333, 0.09611085, 0.73747775, 0.17884378,
        0.12866678, 0.90725255, 0.67241788, 0.94779316, 0.75516832,
        0.56277588, 0.74096592, 0.88334308, 0.08267869, 0.98425163,
        0.07809417, 0.85447412, 0.61777022, 0.06439601, 0.81936819,
        0.29494959, 0.21982498, 0.40658395, 0.76191079, 0.3686963 ]])

In [20]:
KKK=[K,KK]

In [21]:
KKK

[array([[0.70998972, 0.71992364, 0.30119573, 0.20244295, 0.10683678,
         0.79208883, 0.02023982, 0.79557133, 0.7008411 , 0.77696174,
         0.58093416, 0.57751959, 0.30056209, 0.1749741 , 0.29239514,
         0.69407365, 0.49015877, 0.2605282 , 0.7782107 , 0.82409314,
         0.09254306, 0.492695  , 0.8223652 , 0.88640789, 0.59002843,
         0.06482253, 0.03791211, 0.72042701, 0.36633764, 0.45148386,
         0.47161943, 0.93889374, 0.53560761, 0.2406307 , 0.79240599,
         0.99778994, 0.281241  , 0.408207  , 0.46509142, 0.11584683,
         0.08842605, 0.00796081, 0.95978046, 0.95702752, 0.27990984,
         0.89244255, 0.84229321, 0.05528952, 0.68625034, 0.66076211]]),
 array([[0.77975869, 0.93603067, 0.79986745, 0.04857298, 0.73845586,
         0.02749797, 0.87503324, 0.53976727, 0.05298363, 0.3511038 ,
         0.96704053, 0.91207224, 0.0372148 , 0.45494143, 0.64264947,
         0.70155295, 0.62358242, 0.00407979, 0.39574575, 0.71317738,
         0.9508727 , 0.38351257

In [22]:
k=1-KKK[0]*KKK[1]

In [23]:
k.shape

(1, 50)

In [24]:
k

array([[0.44637935, 0.32612939, 0.75908334, 0.99016674, 0.92110575,
        0.97821917, 0.98228948, 0.57057663, 0.96286689, 0.72720578,
        0.43821312, 0.47326041, 0.98881464, 0.92039703, 0.81209242,
        0.51307059, 0.69434561, 0.9989371 , 0.69202642, 0.41227541,
        0.91200334, 0.81104528, 0.98080068, 0.49390766, 0.60960842,
        0.99188596, 0.98420114, 0.93075915, 0.72983414, 0.91925492,
        0.93931825, 0.14818627, 0.63984787, 0.77193187, 0.4016001 ,
        0.43846789, 0.79161001, 0.63941317, 0.96154685, 0.88597757,
        0.99309444, 0.99319769, 0.40707621, 0.93837125, 0.77065078,
        0.73677444, 0.81484291, 0.97752017, 0.47713846, 0.75637945]])

In [27]:
np.sum(k,axis=0)

array([0.44637935, 0.32612939, 0.75908334, 0.99016674, 0.92110575,
       0.97821917, 0.98228948, 0.57057663, 0.96286689, 0.72720578,
       0.43821312, 0.47326041, 0.98881464, 0.92039703, 0.81209242,
       0.51307059, 0.69434561, 0.9989371 , 0.69202642, 0.41227541,
       0.91200334, 0.81104528, 0.98080068, 0.49390766, 0.60960842,
       0.99188596, 0.98420114, 0.93075915, 0.72983414, 0.91925492,
       0.93931825, 0.14818627, 0.63984787, 0.77193187, 0.4016001 ,
       0.43846789, 0.79161001, 0.63941317, 0.96154685, 0.88597757,
       0.99309444, 0.99319769, 0.40707621, 0.93837125, 0.77065078,
       0.73677444, 0.81484291, 0.97752017, 0.47713846, 0.75637945])

In [29]:
KK.shape

(1, 50)